In [ ]:
!pip install matplotlib

In [ ]:
import syft as sy
%matplotlib inline
duet = sy.join_duet(loopback=True)

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        
        self.lin =self.torch_ref.nn.Linear(5,40)
    def forward(self, x):
        x = self.lin(x)
        return x

In [ ]:
import torch
model1 = SyNet(torch)
model1_ptr = model1.send(duet)

In [ ]:
opt1 = duet.torch.optim.Adam(params=model1_ptr.parameters(),lr=0.01)

In [ ]:
duet.store.pandas

In [ ]:
data_pointer = duet.store[0]

In [ ]:
import numpy as np

In [ ]:

model2 = torch.nn.Linear(40,5)
opt2 = torch.optim.Adam(params=model2.parameters(), lr=0.01)

target = torch.tensor([

[299.97,328.96,293.81,262.3,102.16],[370,430.45,409.42,362.67,300],[310,320,271.45,231.18,258.52],
[223.33,256.26,141.63,258.9,464.03],[435.6,362.03,315.17,310.45,210.03],[226.34,241.52,137.47,182.39,165.67],
[172.84,209.42,529.27,369.55,279.7],[256.29,366.23,319.29,214.81,144.73],[177.06,232.17,363.65,379.94,420.3],
[321.29,311.8,195.06,203.48,257.57],[234.74,270.2,227.55,242.53,332.94],[308.1,328,575.26,388.77,311.26],
[124.23,99.21,130.35,172.7,251.16],[202.73,339.23,127.48,94.33,86.35],[149,192.61,157.29,222.03,336.13],
[323.8,218.16,307.93,465.94,615.35],[527.4,269,163.33,181.61,251.65],[290.61,276.32,221.1,35.9,271.77],
[393.32,454.23,604.63,521.81,298.9],[197.61,226.77,152.93,122.39,87.3],[89.9,275.9,494.06,462.97,427.9],
[377.23,325.57,280.84,339.52,290.07],[271.29,188.93,225.45,178.53,306.55],[427.65,441.93,399.68,505.03,435.87],
[240.13,235.83,284.61,244.2,291.1],[302.47,289.32,307.52,279.17,287.68],[299.7,283.84,265.29,239.32,258.77],
[261.73,233.39,212.17,480.71,445.9],[211.8,387.1,242.77,224,229.52],[269.57,267.55,273.13,243.35,253.07],
[283.03,312.9,307.7,502.55,307.73],[270.03,416.65,392.79,215.74,156.1],[209.65,187.87,261.81,275.1,286.17],
[388.68,344.87,268.9,214.94,214.14],[197.26,192.77,175.35,184.43,414.16],[428.65,132.47,379.29,281.5,296],
[106.97,141.39,152.55,240.4,407.48],[264.65,368.96,366.42,294.37,181.16],[165.23,281.29,374.29,104.7,414.48],
[208,141.81,256.61,290.68,299.03]])


target.shape
#target.type

In [ ]:
%%time
import matplotlib.pyplot as plt
#epoch = 200
losses = []
predictions = []
for iter in range(250):
    opt1.zero_grad()
    opt2.zero_grad()
    
    activation_ptr = model1_ptr(data_pointer)
    activation = activation_ptr.clone().get(request_block=True)
    
    pred = model2(activation)
    
    #target = target.detach().numpy()
    loss = ((pred - target)**2).sum()
   
    loss.backward()
    
    grad_ptr = activation.grad.clone().send(duet)
    activation_ptr.backward(grad_ptr)
    
    opt1.step()
    opt2.step()
    losses.append(loss.tolist())
    predictions.append(pred.tolist())
    
    print("losses: ",losses)
    print('Predictions =',pred)


In [ ]:
%%time
import matplotlib.pyplot as plt
#%matplotlib inline
#pred = pred.detach().numpy()
#pred = pred.numpy()
#loss = loss.detach().numpy()
# print("lost",losses)
# print("list",losses[0].tolist())
# print("pred", predictions[0].tolist())
# print("pred", predictions[0].ravel().tolist())
#model2 = model2.detach().numpy()
#target = target.detach().numpy()
plt.title('Diameter of Bulb-NFT Hydroponics')
plt.plot(losses,color='r')
plt.ylabel("Loss")
plt.xlabel("epoch")
plt.show()